In [1]:
import time

install_start=time.time()
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
import sys, os, h5py
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from numpy.random import seed
import tensorflow as tf
install_end=time.time()
print(f"Runtime of the installation piece in seconds is {(install_end - install_start)}")
print(f"Runtime of the installation piece in minutes is {(install_end - install_start)/60}")

Using TensorFlow backend.


Runtime of the installation piece in seconds is 2.5672640800476074
Runtime of the installation piece in minutes is 0.04278773466746012


In [2]:
dataprep_start=time.time()

#Input data processing
input_data = pd.read_csv("~/projects/sdd/data/Sensorless_drive_diagnosis.txt",  delim_whitespace=True, header=None)
#print(input_data.shape)
#header_names = ['col' + str(i) for i in range(input_data.shape[1]-1)]
#header_names.append('label')
#input_data.set_axis(header_names, axis=1, inplace=True)
#display(input_data.describe())

#divide the data into x and y
X=input_data.iloc[:,:-1]
Y=to_categorical(input_data.iloc[:,-1])
#print(X.shape, Y.shape)

seed(94)
#create test/train splits of the data
X_full_train, X_test, Y_full_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

#the states are balanced, but require a transformation bc values are varied
#normalize the values
mean=X_full_train.mean(axis=0)
X_full_train-=mean
std=X_full_train.std(axis=0)
X_full_train /=std

#make sure you scale the test by the training values too
X_test -=mean
X_test /=std

#create an additional split for validation in hyperparamter tuning
X_train, X_valid, Y_train, Y_valid = train_test_split(X_full_train, Y_full_train, test_size=0.15, random_state=42)
#print(X_train[10:15])
dataprep_end=time.time()
print(f"Runtime of the data prep piece in seconds is {(dataprep_end - dataprep_start)}")
print(f"Runtime of the data prep piece in minutes is {(dataprep_end - dataprep_start)/60}")

Runtime of the data prep piece in seconds is 0.9107966423034668
Runtime of the data prep piece in minutes is 0.015179944038391114


In [3]:
#train the model (this code uses build_model() and (train_model() functions)
seed(94)
learn_start=time.time()
#build the model
input_dim=X_train.shape[1]
output_dim=Y_train.shape[1]
model = models.Sequential()
model.add(Dense(input_dim, input_dim=input_dim, activation='relu'))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(input_dim//2, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))
#adjust the default learning rate of adam
opt=optimizers.adam(lr=0.0001)
#compile the model
model.compile(loss='categorical_crossentropy',
        optimizer=opt, metrics=['accuracy'])
#fit the model to training data
history = model.fit(x=X_full_train, y=Y_full_train, epochs=900, verbose=1)   
learn_end=time.time()
print(f"Runtime of learning in seconds is {(learn_end - learn_start)}")
print(f"Runtime of learning in minutes is {(learn_end - learn_start)/60}")

Epoch 1/900
46807/46807 [==============================] - 4s 84us/step - loss: 2.2414 - accuracy: 0.1897
Epoch 2/900
46807/46807 [==============================] - 4s 81us/step - loss: 1.6074 - accuracy: 0.3761
Epoch 3/900
46807/46807 [==============================] - 4s 82us/step - loss: 1.2040 - accuracy: 0.5065
Epoch 4/900
46807/46807 [==============================] - 4s 84us/step - loss: 0.9435 - accuracy: 0.6143
Epoch 5/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.7697 - accuracy: 0.6848
Epoch 6/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.6740 - accuracy: 0.7294
Epoch 7/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.5898 - accuracy: 0.7653
Epoch 8/900
46807/46807 [==============================] - 4s 85us/step - loss: 0.5341 - accuracy: 0.7888
Epoch 9/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.4857 - accuracy: 0.8093
Epoch 10/900
46807/46807 [====================

46807/46807 [==============================] - 4s 81us/step - loss: 0.0366 - accuracy: 0.9889
Epoch 154/900
46807/46807 [==============================] - 4s 80us/step - loss: 0.0357 - accuracy: 0.9890
Epoch 155/900
46807/46807 [==============================] - 4s 84us/step - loss: 0.0343 - accuracy: 0.9895
Epoch 156/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0341 - accuracy: 0.9897
Epoch 157/900
46807/46807 [==============================] - 4s 81us/step - loss: 0.0380 - accuracy: 0.9881
Epoch 158/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0331 - accuracy: 0.9895
Epoch 159/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0335 - accuracy: 0.9897
Epoch 160/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0352 - accuracy: 0.9894
Epoch 161/900
46807/46807 [==============================] - 4s 80us/step - loss: 0.0344 - accuracy: 0.9887
Epoch 162/900
46807/46807 [===============

Epoch 229/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0256 - accuracy: 0.9926
Epoch 230/900
46807/46807 [==============================] - 4s 84us/step - loss: 0.0256 - accuracy: 0.9921
Epoch 231/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0237 - accuracy: 0.9927
Epoch 232/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0245 - accuracy: 0.9923
Epoch 233/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0250 - accuracy: 0.9922
Epoch 234/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0246 - accuracy: 0.9923
Epoch 235/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0258 - accuracy: 0.9924
Epoch 236/900
46807/46807 [==============================] - 4s 81us/step - loss: 0.0242 - accuracy: 0.9924
Epoch 237/900
46807/46807 [==============================] - 4s 81us/step - loss: 0.0240 - accuracy: 0.9929
Epoch 238/900
46807/46807 [=

Epoch 380/900
46807/46807 [==============================] - 4s 77us/step - loss: 0.0163 - accuracy: 0.9949
Epoch 381/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0182 - accuracy: 0.9946
Epoch 382/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0177 - accuracy: 0.9947
Epoch 383/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0170 - accuracy: 0.9944
Epoch 384/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0175 - accuracy: 0.9944
Epoch 385/900
46807/46807 [==============================] - 4s 77us/step - loss: 0.0156 - accuracy: 0.9953
Epoch 386/900
46807/46807 [==============================] - 4s 77us/step - loss: 0.0165 - accuracy: 0.9950
Epoch 387/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0156 - accuracy: 0.9954
Epoch 388/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0189 - accuracy: 0.9946
Epoch 389/900
46807/46807 [=

Epoch 531/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0141 - accuracy: 0.9958
Epoch 532/900
46807/46807 [==============================] - 3s 72us/step - loss: 0.0138 - accuracy: 0.9958
Epoch 533/900
46807/46807 [==============================] - 4s 75us/step - loss: 0.0128 - accuracy: 0.9956
Epoch 534/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0146 - accuracy: 0.9955
Epoch 535/900
46807/46807 [==============================] - 4s 79us/step - loss: 0.0145 - accuracy: 0.9956
Epoch 536/900
46807/46807 [==============================] - 4s 80us/step - loss: 0.0142 - accuracy: 0.9956
Epoch 537/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0181 - accuracy: 0.9958
Epoch 538/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0151 - accuracy: 0.9955
Epoch 539/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0151 - accuracy: 0.9956
Epoch 540/900
46807/46807 [=

Epoch 682/900
46807/46807 [==============================] - 4s 75us/step - loss: 0.0109 - accuracy: 0.9966
Epoch 683/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0113 - accuracy: 0.9968
Epoch 684/900
46807/46807 [==============================] - 4s 79us/step - loss: 0.0121 - accuracy: 0.9964
Epoch 685/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0108 - accuracy: 0.9967
Epoch 686/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0134 - accuracy: 0.9962
Epoch 687/900
46807/46807 [==============================] - 4s 81us/step - loss: 0.0116 - accuracy: 0.9964
Epoch 688/900
46807/46807 [==============================] - 4s 81us/step - loss: 0.0137 - accuracy: 0.9958
Epoch 689/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0124 - accuracy: 0.9963
Epoch 690/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0114 - accuracy: 0.9963
Epoch 691/900
46807/46807 [=

Epoch 833/900
46807/46807 [==============================] - 4s 81us/step - loss: 0.0098 - accuracy: 0.9968
Epoch 834/900
46807/46807 [==============================] - 4s 78us/step - loss: 0.0128 - accuracy: 0.9963
Epoch 835/900
46807/46807 [==============================] - 4s 76us/step - loss: 0.0125 - accuracy: 0.9964
Epoch 836/900
46807/46807 [==============================] - 4s 82us/step - loss: 0.0109 - accuracy: 0.9966
Epoch 837/900
46807/46807 [==============================] - 4s 83us/step - loss: 0.0111 - accuracy: 0.9968
Epoch 838/900
46807/46807 [==============================] - 4s 78us/step - loss: 0.0115 - accuracy: 0.9967
Epoch 839/900
46807/46807 [==============================] - 4s 81us/step - loss: 0.0119 - accuracy: 0.9963
Epoch 840/900
46807/46807 [==============================] - 4s 79us/step - loss: 0.0101 - accuracy: 0.9965
Epoch 841/900
46807/46807 [==============================] - 4s 77us/step - loss: 0.0115 - accuracy: 0.9965
Epoch 842/900
46807/46807 [=

In [4]:
predict_start=time.time()
# evaluate the model on the test set
results = model.evaluate(X_test, Y_test)
predict_end=time.time()
print(f"Runtime of learning in seconds is {(predict_end - predict_start)}")
print(f"Runtime of learning in minutes is {(predict_end - predict_start)/60}")
print(" test set %s: %.2f%%" % (model.metrics_names[1], results[1]*100))


11702/11702 [==============================] - 0s 31us/step
Runtime of learning in seconds is 0.44208574295043945
Runtime of learning in minutes is 0.0073680957158406574
 test set accuracy: 99.56%
